# Download data

In [2]:
import os
import requests
import zipfile

In [3]:
def ensure_directory_exists(directory):
    """Ensure the directory exists. Create it if not."""
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
        return False
    else:
        print(f"Directory '{directory}' already exists.")
        return True

In [4]:
def download_file_if_not_exists(file_path, url):
    """Download a file if it does not exist."""
    if not os.path.exists(file_path):
        response = requests.get(url, stream=True)
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Downloaded '{file_path}'.")
    else:
        print(f"File '{file_path}' already exists.")

In [5]:
def unzip(zip_path, extract_to):
    """Unzip a file and delete the .zip file afterward."""
    if zipfile.is_zipfile(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Extracted '{zip_path}' to '{extract_to}'.")
    else:
        print(f"'{zip_path}' is not a valid zip file.")

In [6]:
# Main execution
directory = "datasets"
zip_file_path = os.path.join(directory, "votacao_candidato_munzona_2022.zip")
download_url = "https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_candidato_munzona/votacao_candidato_munzona_2022.zip"  # Replace with the actual URL
directory_out = zip_file_path.split(".")[0]

# Ensure directory exists
ensure_directory_exists(directory)
ensure_directory_exists(directory_out)

# Download the file if it doesn't exist
download_file_if_not_exists(zip_file_path, download_url)

# Unzip the file and delete the .zip
unzip(zip_file_path, directory_out)

Directory 'datasets' already exists.
Directory 'datasets\votacao_candidato_munzona_2022' already exists.
File 'datasets\votacao_candidato_munzona_2022.zip' already exists.
Extracted 'datasets\votacao_candidato_munzona_2022.zip' to 'datasets\votacao_candidato_munzona_2022'.


# Carregando os dados

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
with open("datasets/votacao_candidato_munzona_2022/votacao_candidato_munzona_2022_BA.csv","r") as file:
    text = file.read()

In [10]:
data = []
for line in text.split("\n"):
    campos = line.split(";")
    
    if len(campos) == 50:
        data.append(campos)
        
columns = data[0]
columns = [i[1:-1] for i in columns]
data = data[1:]
data = pd.DataFrame(data,columns = columns)

In [11]:
columns = data.columns 
for i in columns:
    if len(data[i].unique()) == 1:
        data.drop(i,axis=1,inplace=True)

# Passar nulo para np.nan

In [13]:
nulo = data['SG_FEDERACAO'].values[0]

In [14]:
data.replace(nulo,value=pd.NA,inplace=True)

In [15]:
data.isnull().mean(axis=0)

NR_TURNO                     0.000000
CD_ELEICAO                   0.000000
DT_ELEICAO                   0.000000
CD_MUNICIPIO                 0.000000
NM_MUNICIPIO                 0.000000
NR_ZONA                      0.000000
CD_CARGO                     0.000000
DS_CARGO                     0.000000
SQ_CANDIDATO                 0.000000
NR_CANDIDATO                 0.000000
NM_CANDIDATO                 0.000000
NM_URNA_CANDIDATO            0.000000
NM_SOCIAL_CANDIDATO          0.997384
CD_SITUACAO_CANDIDATURA      0.000000
DS_SITUACAO_CANDIDATURA      0.000000
CD_DETALHE_SITUACAO_CAND     0.000000
DS_DETALHE_SITUACAO_CAND     0.000000
TP_AGREMIACAO                0.000000
NR_PARTIDO                   0.000000
SG_PARTIDO                   0.000000
NM_PARTIDO                   0.000000
NR_FEDERACAO                 0.000000
NM_FEDERACAO                 0.839765
SG_FEDERACAO                 0.839765
DS_COMPOSICAO_FEDERACAO      0.839765
SQ_COLIGACAO                 0.000000
NM_COLIGACAO

# Tirar alguns redudantes

In [17]:
for i in data.columns:
    if len(data[i].unique()) < 10:
        print(data[i].unique())
        print(i)
        print()
    

['1' '2']
NR_TURNO

['546' '547']
CD_ELEICAO

['"02/10/2022"' '"30/10/2022"']
DT_ELEICAO

['7' '3' '6' '5']
CD_CARGO

['"Deputado Estadual"' '"Governador"' '"Deputado Federal"' '"Senador"']
DS_CARGO

[<NA> '"MARIA FERNANDA ABREU DOROTEIA"' '"MARCELO EVANGELISTA"'
 '"LEO KRET DO BRASIL"' '"CAMILA PARKER CONCEIÇÃO DE OLIVEIRA"']
NM_SOCIAL_CANDIDATO

['12' '3']
CD_SITUACAO_CANDIDATURA

['"APTO"' '"INAPTO"']
DS_SITUACAO_CANDIDATURA

['2' '14']
CD_DETALHE_SITUACAO_CAND

['"DEFERIDO"' '"INDEFERIDO"']
DS_DETALHE_SITUACAO_CAND

['"PARTIDO ISOLADO"' '"COLIGAÇÃO"' '"FEDERAÇÃO"']
TP_AGREMIACAO

['-1' '3' '2' '1']
NR_FEDERACAO

[<NA> '"Federação PSOL REDE"'
 '"Federação Brasil da Esperança - FE BRASIL"'
 '"Federação PSDB Cidadania"']
NM_FEDERACAO

[<NA> '"PSOL/REDE"' '"PT/PC do B/PV"' '"PSDB/CIDADANIA"']
SG_FEDERACAO

[<NA> '"PSOL / REDE"' '"PC do B / PT / PV"' '"CIDADANIA / PSDB"']
DS_COMPOSICAO_FEDERACAO

['"PARTIDO ISOLADO"' '"AGORA É A VEZ DO POVO"' '"FEDERAÇÃO"'
 '"JUNTOS VAMOS GOVERNAR"' '"B

In [18]:
redudante = ['CD_ELEICAO','DT_ELEICAO','NR_FEDERACAO','CD_CARGO']
data.drop(redudante,axis=1,inplace=True)

# Corrigir strings

In [20]:
def remove_aspas(x):
    if isinstance(x,str) == False:
        return x
    x = [i for i in x if i !="\""]
    return ''.join(x)
    
for i in data.columns:
    data[i] = data[i].apply(remove_aspas)

# Tipos

In [22]:
for i in data.columns:
    try:
        data[i] = data[i].astype(int)
    except:
        pass

# Alinhar municipios

In [24]:
import geopandas

In [25]:
mun = geopandas.read_file('datasets/BA_Municipios_2022/BA_Municipios_2022.dbf')
mun_names = mun['NM_MUN'].apply(lambda x: x.upper()).values
mun

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry
0,2900108,Abaíra,BA,538.677,"POLYGON ((-41.74667 -13.32509, -41.74679 -13.3..."
1,2900207,Abaré,BA,1604.923,"POLYGON ((-39.23481 -8.70419, -39.23427 -8.705..."
2,2900306,Acajutiba,BA,181.475,"POLYGON ((-38.02809 -11.75007, -38.02811 -11.7..."
3,2900355,Adustina,BA,629.099,"POLYGON ((-37.97867 -10.652, -37.97874 -10.652..."
4,2900405,Água Fria,BA,742.775,"POLYGON ((-38.60169 -11.92432, -38.59707 -11.9..."
...,...,...,...,...,...
412,2933307,Vitória da Conquista,BA,3254.186,"POLYGON ((-40.71779 -14.88926, -40.71774 -14.8..."
413,2933406,Wagner,BA,522.370,"POLYGON ((-41.06276 -12.28494, -41.07128 -12.2..."
414,2933455,Wanderley,BA,2920.579,"POLYGON ((-43.82392 -12.12517, -43.82324 -12.1..."
415,2933505,Wenceslau Guimarães,BA,655.057,"POLYGON ((-39.49336 -13.60308, -39.49335 -13.6..."


In [26]:
for i in data['NM_MUNICIPIO'].unique():
    if i not in mun_names:
        print(i)

SANTO ESTEVÃO
DIAS D ÁVILA
CAMACÃ
CAEM


In [27]:
len(data['NM_MUNICIPIO'].unique())

417

In [28]:
data.replace('CAMACÃ', 'CAMACAN',inplace=True)
data.replace('SANTO ESTEVÃO', 'SANTO ESTÊVÃO',inplace=True)
data.replace('CAEM', 'CAÉM',inplace=True)
data.replace('DIAS D ÁVILA', 'DIAS D\'ÁVILA',inplace=True)

In [29]:
mun_dict = {nm_mun.upper():cd_mun for cd_mun,nm_mun in mun[['CD_MUN','NM_MUN']].values }

In [30]:
data['CD_MUN'] = data['NM_MUNICIPIO'].apply(lambda x: mun_dict[x])
data['NM_MUN'] = data['NM_MUNICIPIO']

In [31]:
data.columns

Index(['NR_TURNO', 'CD_MUNICIPIO', 'NM_MUNICIPIO', 'NR_ZONA', 'DS_CARGO',
       'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
       'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'NM_FEDERACAO', 'SG_FEDERACAO', 'DS_COMPOSICAO_FEDERACAO',
       'SQ_COLIGACAO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO',
       'QT_VOTOS_NOMINAIS', 'NM_TIPO_DESTINACAO_VOTOS',
       'QT_VOTOS_NOMINAIS_VALIDOS', 'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO',
       'CD_MUN', 'NM_MUN'],
      dtype='object')

In [32]:
data.drop(['CD_MUNICIPIO','NM_MUNICIPIO'],axis=1,inplace=True)

# SQL

In [34]:
import sqlite3

In [35]:
db_path = 'sql/eleicao.db' 
connection = sqlite3.connect(db_path)

# votos_candidato

In [37]:
data.columns

Index(['NR_TURNO', 'NR_ZONA', 'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO',
       'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'NM_SOCIAL_CANDIDATO',
       'CD_SITUACAO_CANDIDATURA', 'DS_SITUACAO_CANDIDATURA',
       'CD_DETALHE_SITUACAO_CAND', 'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO',
       'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'NM_FEDERACAO',
       'SG_FEDERACAO', 'DS_COMPOSICAO_FEDERACAO', 'SQ_COLIGACAO',
       'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO', 'QT_VOTOS_NOMINAIS',
       'NM_TIPO_DESTINACAO_VOTOS', 'QT_VOTOS_NOMINAIS_VALIDOS',
       'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO', 'CD_MUN', 'NM_MUN'],
      dtype='object')

In [38]:
votos_candidato = data[['NR_TURNO','NR_ZONA','DS_CARGO','NR_CANDIDATO','NM_CANDIDATO','NM_URNA_CANDIDATO',
      'NM_SOCIAL_CANDIDATO','CD_SITUACAO_CANDIDATURA','DS_SITUACAO_CANDIDATURA','NR_PARTIDO','SG_FEDERACAO','SQ_COLIGACAO','QT_VOTOS_NOMINAIS',
     'NM_TIPO_DESTINACAO_VOTOS','QT_VOTOS_NOMINAIS_VALIDOS','CD_SIT_TOT_TURNO','DS_SIT_TOT_TURNO','CD_MUN']]

In [39]:
# Converter o DataFrame em uma tabela SQL
table_name = 'votos_candidato'
votos_candidato.to_sql(table_name, connection, if_exists='replace', index=False)

688050

In [40]:
# Fechar a conexão
connection.close()